# Ler dados das estações automaticas do INMET

In [3]:
#importar as bibliotecas
import pandas as pd
import sqlite3 as sql
import datetime as dt

# Buscar a situação das estacaoes automaticas


# TODO: implementar a leitura do codigo da estacao a partir do arquivo CSV

Link tutorial json with pans
https://stackabuse.com/reading-and-writing-json-files-in-python-with-pandas/#:~:text=To%20read%20a%20JSON%20file,form%20of%20columns%20and%20rows.

# Ler dados a partir do banco de dados pynmet

link tutorial sqlite
https://towardsdatascience.com/python-pandas-and-sqlite-a0e2c052456f

In [1]:
# codigo da estacao que desejamos atualizar
codigo="A803"

In [4]:
# baixar todas as datas
conn = sql.connect("inmet.db")
query = """
SELECT TIME FROM {codigo}
""".format(codigo=codigo)
db = pd.read_sql(query,conn)

# Buscar a data do ultimo registro cadastrado no banco de dados

In [5]:
#definir TIME como indice
db.set_index('TIME', inplace=True)

In [6]:
if len(db) == 0:
    print('Tabela para a estacao {codigo} nao existe ou está vazia'.format(codigo=codigo))
else:
    print('Tabela para a estacao {codigo} foi encontrada com sucesso'.format(codigo=codigo))

Tabela para a estacao A803 foi encontrada com sucesso


# Checar registros de datas duplicadas

In [7]:
if db[db.index.duplicated()].empty:
    print('Tudo  em ordem na estacao {codigo}'.format(codigo=codigo))
else:
    print('Foram encontrados dados duplicados. Implementar solucao!!')

Tudo  em ordem na estacao A803


In [8]:
# formato de data para buscar na API do INMET
fmt = "%Y-%m-%d"

# ultimo registro cadastrdo da estacao
data_u = db.index.max().split(' ')[0]

#converte data do formato texto para o tipo datetime. Util para fazer operacoes de calculo com  data
data_u = dt.datetime(int(data_u.split('-')[0]),int(data_u.split('-')[1]),int(data_u.split('-')[2]))

print(data_u.strftime(fmt))


2021-05-26


# buscar um periodo entre a ultimo registro e o dia de hoje
# * até o momento não existe um limitação quanto ao número de dias

# definir alguns limites para o periodo.
Não deve exceder a data final não deve exceder o dia de hoje

In [9]:
data_h = dt.datetime.now()
periodo = data_h - data_u
print('Ultima data registrada: {data}'.format(data = data_u.strftime(fmt)))

if (data_h.strftime(fmt) == (data_u.strftime(fmt))):
    print('Estacao {codigo} esta atualizada'.format(codigo = codigo))
    atualizar = False
else:
    print("intervalo de busca de {dias} dias".format(dias=periodo.days))
    atualizar = True

Ultima data registrada: 2021-05-26
intervalo de busca de 1 dias


In [10]:
data_f = data_u + dt.timedelta(days=periodo.days)
print(data_f.strftime(fmt))

2021-05-27


In [11]:
data_i = data_u.strftime(fmt)
data_f = data_f.strftime(fmt)

url = "https://apitempo.inmet.gov.br/estacao/"+data_i+"/"+data_f+"/"+codigo

if atualizar:
    print(url)
else:
    print('Tudo esta em ordem com a estacao {codigo}'.format(codigo=codigo))

https://apitempo.inmet.gov.br/estacao/2021-05-26/2021-05-27/A803


# realizar a consulta pela API e criar o dateframe com o resultdo

In [12]:
if atualizar:
    df = pd.read_json(url)
else:
    print('Nada a fazer. A estacao {codigo} esta atualizada'.format(codigo=codigo))

# Seguir os passos do 1 ate o 8 caso seja necessário atualizar.

# 1. Renomear as colunas compativel com o padrão do Pynmet
(https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas)

In [13]:
df = df.rename(columns = {'TEM_INS':'Temperatura', 'TEM_MAX':'Temperatura_max', 'TEM_MIN':'Temperatura_min', 'UMD_INS':'Umidade',
          'UMD_MAX':'Umidade_max', 'UMD_MIN':'Umidade_min', 'PTO_INS':'Ponto_orvalho',
          'PTO_MAX':'Ponto_orvalho_max', 'PTO_MIN':'Ponto_orvalho_min', 'PRE_INS':'Pressao',
          'PRE_MAX':'Pressao_max', 'PRE_MIN':'Pressao_min', 'VEN_VEL':'Vento_velocidade', 'VEN_DIR':'Vento_direcao',
          'VEN_RAJ':'Vento_rajada', 'RAD_GLO':'Radiacao', 'CHUVA':'Precipitacao'})

# 2. Criar datetime [2019/05/29 13:00:00 ] com as colunas 'DT_MEDICAO' e 'HR_MEDICAO'  


# 3. Definir o datetime criado como index do dataframe

In [14]:
# ajuste da hora para ficar no padrao %H Ex 1 ao inves de 100
df['HR_MEDICAO'] = df['HR_MEDICAO'].apply(lambda x: x/100).astype(int)
#converte a data e hora para o tipo texto
df[['DT_MEDICAO','HR_MEDICAO']] = df[['DT_MEDICAO','HR_MEDICAO']].astype(str)
#Cria uma lista com o datetime no paadrao %Y-%m-%d%H 2020-05-01 13:00:00
data = pd.to_datetime(df['DT_MEDICAO'] + df['HR_MEDICAO'], format="%Y-%m-%d%H")
#Associa o datetime ao indice dos dados baixados
df.set_index(data, inplace=True)

# 4. Apagar colunas que não precisamos 
['DT_MEDICAO','DC_NOME','VL_LATITUDE','UF','CD_ESTACAO','VL_LONGITUDE','HR_MEDICAO']

In [15]:
df.drop(['DT_MEDICAO','DC_NOME','VL_LATITUDE','UF','CD_ESTACAO','VL_LONGITUDE','HR_MEDICAO'], axis=1, inplace=True)

# 5. Reordenar as colunas na sequencia armazenada no inmet.db
(https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns)

In [16]:
df = df[['Temperatura', 'Temperatura_max', 'Temperatura_min', 'Umidade', 'Umidade_max', 'Umidade_min', 'Ponto_orvalho', 'Ponto_orvalho_max', 'Ponto_orvalho_min', 'Pressao', 'Pressao_max', 'Pressao_min', 'Vento_velocidade', 'Vento_direcao', 'Vento_rajada', 'Radiacao', 'Precipitacao']]

# 6. verificar se o periodo baixado do INMET já está cadastrado no banco de dados do pynmet

# 7. Remover linhas caso todos os valores sejam NA

In [17]:
# se no banco de dados a ultima informacao gravada foi a leitura das 13h do dia 20/05/2020,
# então apenas devera mostrar as leituras a partir das 14h em diante do dia 20/05/2020.
# evitar duplicar informações salvas no db
dados = df[~df.index.isin(db.index)]

In [18]:
# elimina as linhas da data de hj que não tem cadastro no INMET.
dados = dados.dropna(how='all')

# Gravar as informações no banco de dados
# Backup do arquivo antes de fazer alterações
# Revisar os dados antes de salvar!!!!

In [19]:
if len(dados) ==0:
    gravar = False
    print ('A estacao esta inoperante durante o periodo solicitado: {data1} - {data2}'.format(data1=data_i,data2=data_f))
else:
    gravar= True

dados.head(-5)

,Temperatura,Temperatura_max,Temperatura_min,Umidade,Umidade_max,Umidade_min,Ponto_orvalho,Ponto_orvalho_max,Ponto_orvalho_min,Pressao,Pressao_max,Pressao_min,Vento_velocidade,Vento_direcao,Vento_rajada,Radiacao,Precipitacao
2021-05-26 13:00:00,12.1,12.1,9.4,86.0,94.0,83.0,9.8,10.1,8.4,1009.2,1009.3,1008.7,NaN,NaN,NaN,1253.802,0.0
2021-05-26 14:00:00,16.0,16.6,12.1,70.0,86.0,67.0,10.6,11.3,9.7,1008.6,1009.2,1008.4,NaN,NaN,NaN,1769.084,0.0
2021-05-26 15:00:00,17.9,17.9,15.8,60.0,72.0,59.0,10.1,11.4,9.6,1008.0,1008.8,1008.0,NaN,NaN,NaN,2108.258,0.0
2021-05-26 16:00:00,18.7,19.3,17.7,54.0,63.0,52.0,9.2,10.7,9.0,1006.8,1008.0,1006.8,NaN,NaN,NaN,2240.304,0.0
2021-05-26 17:00:00,19.5,19.7,18.7,53.0,59.0,52.0,9.8,11.0,9.2,1006.0,1006.8,1006.0,NaN,NaN,NaN,2138.390,0.0
2021-05-26 18:00:00,20.3,21.2,19.4,52.0,56.0,49.0,10.1,10.9,9.6,1005.5,1006.0,1005.4,NaN,NaN,NaN,1815.327,0.0
2021-05-26 19:00:00,20.3,20.9,19.7,53.0,56.0,51.0,10.4,10.9,9.4,1004.5,1005.5,1004.5,NaN,NaN,NaN,1322.206,0.0
2021-05-26 20:00:00,18.9,20.7,18.9,58.0,59.0,51.0,10.4,10.9,10.0,1004.4,1004.6,1004.4,NaN,NaN,NaN,686.667,0.0
2021-05-26 21:00:00,15.0,18.9,15.0,78.0,78.0,58.0,11.1,11.1,10.1,1004.7,1004.7,1004.4,NaN,NaN,NaN,67.538,0.0
2021-05-26 22:00:00,12.0,15.0,12.0,92.0,92.0,74.0,10.7,11.4,10.1,1004.8,1004.8,1004.7,NaN,NaN,NaN,-3.600,0.0


# 8. Atualizar o banco de dados com as novas informações

In [20]:
if gravar:
    dados.to_sql(codigo, conn, if_exists='append', index_label='TIME')
    print('Dados da estacao {codigo} atualizada com sucesso!'.format(codigo=codigo))
else:
    print('Não foram feitas alterações na estacao {codigo}.'.format(codigo=codigo))

Dados da estacao A803 atualizada com sucesso!
